In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer

## Combine Scorecard & IPEDS Data

In [2]:
# Get the path to the current folder
current_folder = Path().cwd()

# Import the two CSV files
college_scorecard = pd.read_csv('CollegeScoreCard_2021_22.csv')
ipeds_data = pd.read_csv('IPEDS Data 2022.csv')
fairtest_data = pd.read_csv('FairTest_List.csv')[['InstitutionName', 'CurrentAdmissionsTestingPolicy']]

# Merge the DataFrames on the InstitutionName column
merged_data = pd.merge(college_scorecard, ipeds_data, on='InstitutionName', how='inner')
merged_data = pd.merge(merged_data, fairtest_data, on='InstitutionName', how='left')

# Drop duplicate rows based on the 'InstitutionName' column (keep the first occurrence)
merged_data = merged_data.drop_duplicates(subset=['InstitutionName'], keep='first')

# Display the DataFrame without duplicates
print(merged_data)

                                        InstitutionName        City StateCode  \
0                              Alabama A & M University      Normal        AL   
1                   University of Alabama at Birmingham  Birmingham        AL   
2                                    Amridge University  Montgomery        AL   
3                   University of Alabama in Huntsville  Huntsville        AL   
4                              Alabama State University  Montgomery        AL   
...                                                 ...         ...       ...   
6385  Pennsylvania State University-Penn State Wilke...      Lehman        PA   
6386      Pennsylvania State University-Penn State York        York        PA   
6387  Pennsylvania State University-Penn State Great...     Malvern        PA   
6388  Pennsylvania State University-Penn State Harri...  Middletown        PA   
6389  Pennsylvania State University-Penn State Brand...       Media        PA   

             ZIP  HIGHDEG  

## Combine and Merge Crime Rate Data

In [3]:
# File paths
files = [
    "crime_40_60.csv",
    "crime_60_100.csv",
    "crime_100_250.csv",
    "crime_250_plus.csv"
]

# Import all files and concatenate into one DataFrame
crime_dataframes = [pd.read_csv(file) for file in files]
crime_data = pd.concat(crime_dataframes, ignore_index=True)

# Dictionary of state names to abbreviations
us_state_to_abbrev = {
    "Alabama": "AL", "Alaska": "AK", "Arizona": "AZ", "Arkansas": "AR", "California": "CA",
    "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "Florida": "FL", "Georgia": "GA",
    "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA",
    "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Maryland": "MD",
    "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS",
    "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH",
    "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC",
    "North Dakota": "ND", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Pennsylvania": "PA",
    "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN",
    "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virginia": "VA", "Washington": "WA",
    "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY", "District of Columbia": "DC",
    "American Samoa": "AS", "Guam": "GU", "Northern Mariana Islands": "MP", "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM", "Virgin Islands, U.S.": "VI"
}

# Convert full state names to abbreviations
def convert_state_to_abbrev(state_name):
    """Converts a full state name to its abbreviation."""
    return us_state_to_abbrev.get(state_name, state_name)  # Keeps the original name if not found

# Apply the conversion
crime_data['state_abbrev'] = crime_data['states'].apply(convert_state_to_abbrev)

# List of columns to exclude from conversion
exclude_columns = ['states', 'cities', 'state_abbrev']

# Remove commas from the population column
crime_data['population'] = crime_data['population'].str.replace(',', '', regex=False)

# Select all columns except for the ones in 'exclude_columns' and convert them to numeric
crime_data = crime_data.apply(lambda x: pd.to_numeric(x, errors='coerce') if x.name not in exclude_columns else x)

# List of columns that represent different types of crimes
crime_columns = ['violent_crime', 'murder', 'rape', 'robbery', 'agrv_assault',
                 'prop_crime', 'burglary', 'larceny', 'vehicle_theft', 'arson']

# Calculate total crime as the sum of the crime-related columns
crime_data['total_crime'] = crime_data[crime_columns].sum(axis=1)

# Add a new column for the crime rate per 100,000 residents
crime_data['crime_rate_per_100k'] = (crime_data['total_crime'] / crime_data['population']) * 100_000

# Only keep the necessary columns
crime_data = crime_data[['state_abbrev', 'cities', 'population', 'crime_rate_per_100k']]

# Merge the two DataFrames on 'City' and 'StateCode' (which matches with 'cities' and 'state_abbrev')
merged_with_crime = pd.merge(merged_data, crime_data, 
                             left_on=['City', 'StateCode'], 
                             right_on=['cities', 'state_abbrev'], 
                             how='left')

# Display the resulting DataFrame with the crime rate information
print(merged_with_crime[['InstitutionName', 'City', 'StateCode', 'population', 'crime_rate_per_100k']].head())

# Display the merged DataFrame
print(merged_with_crime.head())

                       InstitutionName        City StateCode  population  \
0             Alabama A & M University      Normal        AL         NaN   
1  University of Alabama at Birmingham  Birmingham        AL    213266.0   
2                   Amridge University  Montgomery        AL    209018.0   
3  University of Alabama in Huntsville  Huntsville        AL    183691.0   
4             Alabama State University  Montgomery        AL    209018.0   

   crime_rate_per_100k  
0                  NaN  
1           940.796939  
2           630.615545  
3          1215.628419  
4           630.615545  
                       InstitutionName        City StateCode         ZIP  \
0             Alabama A & M University      Normal        AL       35762   
1  University of Alabama at Birmingham  Birmingham        AL  35294-0110   
2                   Amridge University  Montgomery        AL  36117-3553   
3  University of Alabama in Huntsville  Huntsville        AL       35899   
4            

## Merge Cost of Living Data

In [4]:
#### Cost of Living ####
# Step 1: Import the advisorsmith_cost_of_living_index.csv file
cost_of_living_df = pd.read_csv('advisorsmith_cost_of_living_index.csv')

# Step 2: Merge the cost of living data with the merged_with_crime DataFrame
merged_final = pd.merge(merged_with_crime, cost_of_living_df, 
                       how='left', 
                       left_on=['City', 'StateCode'], 
                       right_on=['City', 'State'])

# Step 3: Check the result
print(merged_final.head())
########################

                       InstitutionName        City StateCode         ZIP  \
0             Alabama A & M University      Normal        AL       35762   
1  University of Alabama at Birmingham  Birmingham        AL  35294-0110   
2                   Amridge University  Montgomery        AL  36117-3553   
3  University of Alabama in Huntsville  Huntsville        AL       35899   
4             Alabama State University  Montgomery        AL  36104-0271   

   HIGHDEG  Control  ADM_RATE  ADM_RATE_ALL  SATVRMID  SATMTMID  ...  \
0        4        1    0.7160      0.716006     475.0     460.0  ...   
1        4        1    0.8854      0.885352     637.0     644.0  ...   
2        4        2       NaN           NaN       NaN       NaN  ...   
3        4        1    0.7367      0.736724     640.0     655.0  ...   
4        4        1    0.9799      0.979913     488.0     457.0  ...   

   HD2022.Institution size category  HistoricallyBlack  StudentFacultyRatio  \
0                     5,000 - 9

## Data Imputation & Transformation

In [5]:
# Filter out non-degree offering institutions
merged_final = merged_final[merged_final['HIGHDEG'] > 1]

### Institution Size 

In [6]:
# Replace 'Not applicable' and 'Not reported' with NaN
merged_final['InstitutionSize'] = merged_final['HD2022.Institution size category'].replace(
    ['Not applicable', 'Not reported'], np.nan
)

# Define a mapping dictionary for binning
size_mapping = {
    'Under 1,000': 1,
    '1,000 - 4,999': 2,
    '5,000 - 9,999': 3,
    '10,000 - 19,999': 4,
    '20,000 and above': 5
}

# Map the size categories to numerical bins
merged_final['SizeBin'] = merged_final['InstitutionSize'].map(size_mapping).astype(pd.Int64Dtype())

# Display the updated dataframe
print(merged_final[merged_final['SizeBin'].notna()][['InstitutionName', 'InstitutionSize', 'SizeBin']])

                          InstitutionName   InstitutionSize  SizeBin
0                Alabama A & M University     5,000 - 9,999        3
1     University of Alabama at Birmingham  20,000 and above        5
2                      Amridge University       Under 1,000        1
3     University of Alabama in Huntsville     5,000 - 9,999        3
4                Alabama State University     1,000 - 4,999        2
...                                   ...               ...      ...
5798           Florida Academy of Nursing       Under 1,000        1
5800                  Lakewood University       Under 1,000        1
5813             Madera Community College     5,000 - 9,999        3
5821            Great Northern University       Under 1,000        1
5824             Arizona College-Glendale       Under 1,000        1

[3827 rows x 3 columns]


### Test Optionality

In [7]:
# Convert CurrentAdmissionsTestingPolicy to TestOptional
merged_final['TestOptional'] = merged_final['CurrentAdmissionsTestingPolicy'].apply(lambda x: 1 if x in ['Test Optional', 'Test Free'] else 0)

# Display the updated DataFrame
print(merged_final[['InstitutionName', 'TestOptional']])

                          InstitutionName  TestOptional
0                Alabama A & M University             1
1     University of Alabama at Birmingham             1
2                      Amridge University             1
3     University of Alabama in Huntsville             1
4                Alabama State University             1
...                                   ...           ...
5798           Florida Academy of Nursing             0
5800                  Lakewood University             0
5813             Madera Community College             0
5821            Great Northern University             1
5824             Arizona College-Glendale             0

[3857 rows x 2 columns]


### Selectivity

In [8]:
# Separate non-NaN values for binning
non_nan_data = merged_final['ADM_RATE_ALL'].dropna()

# Perform quantile-based binning on non-NaN values
selectivity_binned = pd.qcut(
    non_nan_data,
    q=5,  # Number of quantiles (bins)
    labels=[5, 4, 3, 2, 1]  # Assign labels, 5 = most selective, 1 = least selective
)

# Create a copy of the binned data to preserve indices
selectivity_binned_with_indices = pd.Series(selectivity_binned, index=non_nan_data.index)

# Assign binned values back to the original DataFrame
merged_final['SelectivityBin'] = selectivity_binned_with_indices

# Convert labels to integers while keeping NaN intact
merged_final['SelectivityBin'] = merged_final['SelectivityBin'].astype(pd.Int64Dtype())

# Display institutions with non-NaN Selectivity
print(merged_final[merged_final['SelectivityBin'].notna()][['InstitutionName', 'ADM_RATE_ALL', 'SelectivityBin']])


                          InstitutionName  ADM_RATE_ALL  SelectivityBin
0                Alabama A & M University      0.716006               4
1     University of Alabama at Birmingham      0.885352               2
3     University of Alabama in Huntsville      0.736724               3
4                Alabama State University      0.979913               1
5               The University of Alabama      0.789043               3
...                                   ...           ...             ...
5785                  Daybreak University      0.014241               5
5792     California Northstate University      0.954819               1
5798           Florida Academy of Nursing      0.308848               5
5821            Great Northern University      1.000000               1
5824             Arizona College-Glendale      0.993776               1

[1864 rows x 3 columns]


### Tuition

In [9]:
# Separate non-NaN values for binning
non_nan_tuition = merged_final['TUITIONFEE_OUT'].dropna()

# Perform quantile-based binning on non-NaN values
tuition_binned = pd.qcut(
    non_nan_tuition,
    q=5,  # Number of quantiles (bins)
    labels=[1, 2, 3, 4, 5]  # Assign labels, 1 = cheapest, 5 = most expensive
)

# Create a copy of the binned data to preserve indices1
tuition_binned_with_indices = pd.Series(tuition_binned, index=non_nan_data.index)

# Assign binned values back to the original DataFrame
merged_final['TuitionBin'] = tuition_binned_with_indices

# Convert labels to integers, keeping NaN intact
merged_final['TuitionBin'] = merged_final['TuitionBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame
print(merged_final[merged_final['TuitionBin'].notna()][['InstitutionName', 'TUITIONFEE_OUT', 'TuitionBin']])

                          InstitutionName  TUITIONFEE_OUT  TuitionBin
0                Alabama A & M University         18634.0           3
1     University of Alabama at Birmingham         20400.0           4
3     University of Alabama in Huntsville         23884.0           4
4                Alabama State University         19396.0           4
5               The University of Alabama         31090.0           4
...                                   ...             ...         ...
5747    California Indian Nations College          7743.0           1
5763    The Pennsylvania State University         36476.0           5
5792     California Northstate University         49793.0           5
5821            Great Northern University         15600.0           3
5824             Arizona College-Glendale         18334.0           3

[1797 rows x 3 columns]


### CrimeRate

In [10]:
# Drop NaN values from the 'crime_rate_per_100k' column
non_nan_crime_rate = merged_final['crime_rate_per_100k'].dropna()

# Perform quantile-based binning on non-NaN values
crime_binned = pd.qcut(
    non_nan_crime_rate,
    q=5,  # Number of quantiles (bins)
    labels=[1, 2, 3, 4, 5]  # Assign labels, 1 = lowest crime rate, 5 = highest crime rate
)

# Create a copy of the binned data to preserve indices
crime_binned_with_indices = pd.Series(crime_binned, index=non_nan_crime_rate.index)

# Assign binned values back to the original DataFrame
merged_final['CrimeRateBin'] = crime_binned_with_indices

# Convert labels to integers while keeping NaN intact
merged_final['CrimeRateBin'] = merged_final['CrimeRateBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame for non-NaN bins
print(merged_final[merged_final['CrimeRateBin'].notna()][['InstitutionName', 'crime_rate_per_100k', 'CrimeRateBin']])


                          InstitutionName  crime_rate_per_100k  CrimeRateBin
1     University of Alabama at Birmingham           940.796939             1
2                      Amridge University           630.615545             1
3     University of Alabama in Huntsville          1215.628419             2
4                Alabama State University           630.615545             1
5               The University of Alabama          1368.771270             3
...                                   ...                  ...           ...
5798           Florida Academy of Nursing          1358.672360             3
5800                  Lakewood University         17344.245100             5
5813             Madera Community College          4818.300529             4
5821            Great Northern University          1073.655633             2
5824             Arizona College-Glendale           647.304472             1

[1131 rows x 3 columns]


### Population

In [11]:
# Drop NaN values from the 'population' column
non_nan_population = merged_final['population'].dropna()

# Perform quantile-based binning on non-NaN values
population_binned = pd.qcut(
    non_nan_population,
    q=5,  # Number of quantiles (bins)
    labels=[1, 2, 3, 4, 5]  # Assign labels, 1 = smallest population, 5 = largest population
)

# Create a copy of the binned data to preserve indices
population_binned_with_indices = pd.Series(population_binned, index=non_nan_population.index)

# Assign binned values back to the original DataFrame
merged_final['PopulationBin'] = population_binned_with_indices

# Convert labels to integers while keeping NaN intact
merged_final['PopulationBin'] = merged_final['PopulationBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame for non-NaN bins
print(merged_final[merged_final['PopulationBin'].notna()][['InstitutionName', 'population', 'PopulationBin']])


                          InstitutionName  population  PopulationBin
1     University of Alabama at Birmingham    213266.0              5
2                      Amridge University    209018.0              5
3     University of Alabama in Huntsville    183691.0              5
4                Alabama State University    209018.0              5
5               The University of Alabama     91973.0              3
...                                   ...         ...            ...
5798           Florida Academy of Nursing    125998.0              4
5800                  Lakewood University     45761.0              1
5813             Madera Community College     62796.0              2
5821            Great Northern University    212163.0              5
5824             Arizona College-Glendale    232997.0              5

[1131 rows x 3 columns]


### Cost of Living

In [12]:
# Drop NaN values from the 'cost of living' column
non_nan_livingcost = merged_final['Cost of Living Index'].dropna()

# Perform quantile-based binning on non-NaN values
livingcost_binned = pd.qcut(
    non_nan_livingcost,
    q=5,  # Number of quantiles (bins)
    labels=[1, 2, 3, 4, 5]  # Assign labels, 1 = smallest population, 5 = largest population
)

# Create a copy of the binned data to preserve indices
livingcost_binned_with_indices = pd.Series(livingcost_binned, index=non_nan_livingcost.index)

# Assign binned values back to the original DataFrame
merged_final['LivingCostBin'] = livingcost_binned_with_indices

# Convert labels to integers while keeping NaN intact
merged_final['LivingCostBin'] = merged_final['LivingCostBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame for non-NaN bins
print(merged_final[merged_final['LivingCostBin'].notna()][['InstitutionName', 'Cost of Living Index', 'LivingCostBin']])


                             InstitutionName  Cost of Living Index  \
1        University of Alabama at Birmingham                  90.7   
2                         Amridge University                  88.5   
3        University of Alabama in Huntsville                  91.3   
4                   Alabama State University                  88.5   
5                  The University of Alabama                  91.0   
...                                      ...                   ...   
5772  San Joaquin Valley College-Santa Maria                 143.0   
5779              Triangle Tech-Chambersburg                  95.7   
5784   Agape College of Business and Science                 105.9   
5813                Madera Community College                 108.9   
5821               Great Northern University                 100.4   

      LivingCostBin  
1                 2  
2                 1  
3                 2  
4                 1  
5                 2  
...             ...  
5772 

### Diversity Index

In [13]:
# Define the columns corresponding to racial shares
racial_columns = ['UGDS_WHITE', 'UGDS_BLACK', 'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR', 'UGDS_UNKN']

# Calculate the Gini-Simpson Index for each row
def calculate_gini_simpson(row):
    # Filter out NaN values and calculate the squared proportions
    proportions = row[racial_columns].dropna()
    squared_proportions = proportions ** 2
    diversity_index = 1 - squared_proportions.sum()
    return diversity_index

# Apply the function to each row of the dataframe and create a new column
merged_final['DiversityIndex'] = merged_final.apply(calculate_gini_simpson, axis=1)

# Display the updated dataframe with the new DiversityIndex
print(merged_final[merged_final['DiversityIndex'].notna()][['InstitutionName', 'DiversityIndex']])

                          InstitutionName  DiversityIndex
0                Alabama A & M University        0.191065
1     University of Alabama at Birmingham        0.646258
2                      Amridge University        0.456386
3     University of Alabama in Huntsville        0.466858
4                Alabama State University        0.142003
...                                   ...             ...
5798           Florida Academy of Nursing        0.290291
5800                  Lakewood University        0.567570
5813             Madera Community College        0.528243
5821            Great Northern University        0.289600
5824             Arizona College-Glendale        0.664906

[3857 rows x 2 columns]


In [14]:
# Separate non-NaN values for binning
non_nan_diversity = merged_final['DiversityIndex'].dropna()

# Perform quantile-based binning on non-NaN values (5 bins)
diversity_binned = pd.qcut(
    non_nan_diversity,
    q=5,  # Number of quantiles (bins)
    labels=[1, 2, 3, 4, 5]  # Assign labels, 1 = lowest diversity, 5 = highest diversity
)

# Create a copy of the binned data to preserve indices
diversity_binned_with_indices = pd.Series(diversity_binned, index=non_nan_diversity.index)

# Assign binned values back to the original DataFrame
merged_final['DiversityIndexBin'] = diversity_binned_with_indices

# Convert labels to integers, keeping NaN intact
merged_final['DiversityIndexBin'] = merged_final['DiversityIndexBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame
print(merged_final[merged_final['DiversityIndexBin'].notna()][['InstitutionName', 'DiversityIndex', 'DiversityIndexBin']])


                          InstitutionName  DiversityIndex  DiversityIndexBin
0                Alabama A & M University        0.191065                  1
1     University of Alabama at Birmingham        0.646258                  4
2                      Amridge University        0.456386                  2
3     University of Alabama in Huntsville        0.466858                  2
4                Alabama State University        0.142003                  1
...                                   ...             ...                ...
5798           Florida Academy of Nursing        0.290291                  1
5800                  Lakewood University        0.567570                  3
5813             Madera Community College        0.528243                  2
5821            Great Northern University        0.289600                  1
5824             Arizona College-Glendale        0.664906                  4

[3857 rows x 3 columns]


### Student to Faculty Ratio

In [15]:
# Drop NaN values from the 'cost of living' column
non_nan_stufacratio = merged_final['StudentFacultyRatio'].dropna()

# Perform quantile-based binning on non-NaN values
stufacratio_binned = pd.qcut(
    non_nan_livingcost,
    q=3,  # Number of quantiles (bins)
    labels=[1, 2, 3]  # Assign labels, 1 = smallest ratio, 3 = largest ratio
)

# Create a copy of the binned data to preserve indices
stufacratio_binned_with_indices = pd.Series(stufacratio_binned, index=non_nan_stufacratio.index)

# Assign binned values back to the original DataFrame
merged_final['StudentFacultyRatioBin'] = stufacratio_binned_with_indices

# Convert labels to integers while keeping NaN intact
merged_final['StudentFacultyRatioBin'] = merged_final['StudentFacultyRatioBin'].astype(pd.Int64Dtype())

# Display the updated DataFrame for non-NaN bins
print(merged_final[merged_final['StudentFacultyRatioBin'].notna()][['InstitutionName', 'StudentFacultyRatio', 'StudentFacultyRatioBin']])


                             InstitutionName  StudentFacultyRatio  \
1        University of Alabama at Birmingham                 18.0   
2                         Amridge University                 12.0   
3        University of Alabama in Huntsville                 17.0   
4                   Alabama State University                 14.0   
5                  The University of Alabama                 19.0   
...                                      ...                  ...   
5772  San Joaquin Valley College-Santa Maria                 18.0   
5779              Triangle Tech-Chambersburg                  9.0   
5784   Agape College of Business and Science                  7.0   
5813                Madera Community College                 31.0   
5821               Great Northern University                  2.0   

      StudentFacultyRatioBin  
1                          1  
2                          1  
3                          1  
4                          1  
5               

## Export

In [16]:
### Save the merged data to a new file 
merged_final.to_csv(current_folder / 'Merged_Data.csv', index=False)

In [17]:
### Subset merged data to get matching data
matching_data = merged_final[['InstitutionName', 'StateCode', 'Control', 'SizeBin', 'SelectivityBin', 'StudentFacultyRatioBin', 'TestOptional', 'SATVRMID', 'SATMTMID', 'ACTMTMID', 'ACTENMID', 'TuitionBin', 'LivingCostBin', 'CrimeRateBin', 'DiversityIndexBin', 'PopulationBin']]
matching_data

,InstitutionName,StateCode,Control,SizeBin,SelectivityBin,StudentFacultyRatioBin,TestOptional,SATVRMID,SATMTMID,ACTMTMID,ACTENMID,TuitionBin,LivingCostBin,CrimeRateBin,DiversityIndexBin,PopulationBin
0,Alabama A & M University,AL,1,3,4,<NA>,1,475.0,460.0,17.0,17.0,3,<NA>,<NA>,1,<NA>
1,University of Alabama at Birmingham,AL,1,5,2,1,1,637.0,644.0,25.0,28.0,4,2,1,4,5
2,Amridge University,AL,2,1,<NA>,1,1,NaN,NaN,NaN,NaN,<NA>,1,1,2,5
3,University of Alabama in Huntsville,AL,1,3,3,1,1,640.0,655.0,26.0,28.0,4,2,2,2,5
4,Alabama State University,AL,1,2,1,1,1,488.0,457.0,17.0,17.0,4,1,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5798,Florida Academy of Nursing,FL,3,1,5,<NA>,0,NaN,NaN,NaN,NaN,<NA>,<NA>,3,1,4
5800,Lakewood University,OH,2,1,<NA>,<NA>,0,NaN,NaN,NaN,NaN,<NA>,<NA>,5,3,1
5813,Madera Community College,CA,1,3,<NA>,3,0,NaN,NaN,NaN,NaN,<NA>,4,4,2,2
5821,Great Northern University,WA,2,1,1,2,1,NaN,NaN,NaN,NaN,3,4,2,1,5


In [19]:
### Save matching data to a new file
matching_data.to_csv(current_folder / 'Matching_Data.csv', index=False)

### MICE Imputation

In [20]:
# Extract non-numerical columns
non_numerical_cols = ["InstitutionName", "StateCode"]
matching_data_non_numerical = matching_data[non_numerical_cols]

# Extract numerical columns
matching_data_numerical = matching_data.drop(columns=non_numerical_cols)

# Step 1: Define the MICE imputer
mice_imputer = IterativeImputer(random_state=42, max_iter=10)

# Step 2: Perform the imputation on numerical data
miced_matching_array = mice_imputer.fit_transform(matching_data_numerical)

# Step 3: Create a new dataframe for the imputed numerical data
miced_matching_numerical = pd.DataFrame(miced_matching_array, columns=matching_data_numerical.columns)

# Round all columns in the dataframe to the nearest integer
miced_matching_numerical = miced_matching_numerical.round(0).astype(int)

# Step 4: Combine the imputed numerical data with the non-numerical columns
miced_matching_data = pd.concat([matching_data_non_numerical.reset_index(drop=True), 
                                 miced_matching_numerical.reset_index(drop=True)], axis=1)

# Step 5: Save the resulting dataframe to a file
miced_matching_data.to_csv('miced_matching_data.csv', index=False)

# Optional: Preview the imputed data
print(miced_matching_data.head())

                       InstitutionName StateCode  Control  SizeBin  \
0             Alabama A & M University        AL        1        3   
1  University of Alabama at Birmingham        AL        1        5   
2                   Amridge University        AL        2        1   
3  University of Alabama in Huntsville        AL        1        3   
4             Alabama State University        AL        1        2   

   SelectivityBin  StudentFacultyRatioBin  TestOptional  SATVRMID  SATMTMID  \
0               4                       1             1       475       460   
1               2                       1             1       637       644   
2               3                       1             1       532       528   
3               3                       1             1       640       655   
4               1                       1             1       488       457   

   ACTMTMID  ACTENMID  TuitionBin  LivingCostBin  CrimeRateBin  \
0        17        17           3     

c:\Users\Hieu Tran\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=28f6e439-b516-405e-ad62-5aa314d601ab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>